# Self-Query Retriever 튜토리얼

이 노트북에서는 LangChain의 Self-Query Retriever를 사용하여 자연어 질의를 통해 메타데이터 기반 검색을 수행하는 방법을 학습합니다.

## 학습 목표
1. Self-Query Retriever의 개념과 동작 원리 이해
2. 메타데이터 태깅을 통한 문서 분류 방법 학습
3. 자연어 질의를 메타데이터 필터와 검색 쿼리로 분해하는 과정 이해
4. 실제 문서 데이터를 활용한 Self-Query 구현 실습

## 목차
1. 데이터 수집 및 전처리
2. 메타데이터 태깅
3. 벡터 스토어 구성
4. Self-Query Retriever 설정
5. 자연어 질의 실습


## 1. 데이터 수집 및 전처리

먼저 arXiv에서 AI 관련 논문들을 수집하여 테스트 데이터로 활용하겠습니다.

### 1.1 필요한 라이브러리 import


In [4]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import pandas as pd
import time
from langchain.schema import Document

In [7]:
# arXiv API 기본 사용법 1: 단일 논문 검색 및 응답 구조 확인
base_url = 'http://export.arxiv.org/api/query?'
query = 'search_query=cat:cs.AI&start=0&max_results=1'
url = base_url + query

print(f"요청 URL: {url}")
print("\n=== arXiv API 응답 (첫 200자) ===")

response = requests.get(url)
if response.status_code == 200:
    print(response.text[:500] + "...")
    print(f"\n응답 크기: {len(response.text)} 문자")
else:
    print(f"API 요청 실패: {response.status_code}")

요청 URL: http://export.arxiv.org/api/query?search_query=cat:cs.AI&start=0&max_results=1

=== arXiv API 응답 (첫 200자) ===
<?xml version="1.0" encoding="UTF-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <link href="http://arxiv.org/api/query?search_query%3Dcat%3Acs.AI%26id_list%3D%26start%3D0%26max_results%3D1" rel="self" type="application/atom+xml"/>
  <title type="html">ArXiv Query: search_query=cat:cs.AI&amp;id_list=&amp;start=0&amp;max_results=1</title>
  <id>http://arxiv.org/api/HQttLwSI9ar2vqfmj4lJ7u3JUas</id>
  <updated>2025-06-10T00:00:00-04:00</updated>
  <opensearch:totalResults xmlns:opensearch="http:...

응답 크기: 2331 문자


In [9]:
print(response.content)

b'<?xml version="1.0" encoding="UTF-8"?>\n<feed xmlns="http://www.w3.org/2005/Atom">\n  <link href="http://arxiv.org/api/query?search_query%3Dcat%3Acs.AI%26id_list%3D%26start%3D0%26max_results%3D1" rel="self" type="application/atom+xml"/>\n  <title type="html">ArXiv Query: search_query=cat:cs.AI&amp;id_list=&amp;start=0&amp;max_results=1</title>\n  <id>http://arxiv.org/api/HQttLwSI9ar2vqfmj4lJ7u3JUas</id>\n  <updated>2025-06-10T00:00:00-04:00</updated>\n  <opensearch:totalResults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">130079</opensearch:totalResults>\n  <opensearch:startIndex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:startIndex>\n  <opensearch:itemsPerPage xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">1</opensearch:itemsPerPage>\n  <entry>\n    <id>http://arxiv.org/abs/cs/9308101v1</id>\n    <updated>1993-08-01T00:00:00Z</updated>\n    <published>1993-08-01T00:00:00Z</published>\n    <title>Dynamic Backtracking</title>\n    <summar

In [10]:
# arXiv API 기본 사용법 2: XML 파싱 및 논문 정보 추출
import xml.etree.ElementTree as ET

# 위에서 받은 응답을 XML로 파싱
try:
    root = ET.fromstring(response.content)
    
    # arXiv API XML 네임스페이스 정의
    ns = {
        'atom': 'http://www.w3.org/2005/Atom',
        'arxiv': 'http://arxiv.org/schemas/atom'
    }
    
    print("=== XML 구조 분석 ===")
    print(f"루트 태그: {root.tag}")
    print(f"총 entry 개수: {len(root.findall('atom:entry', ns))}")
    
    # 첫 번째 논문 정보 추출
    first_entry = root.find('atom:entry', ns)
    if first_entry is not None:
        title = first_entry.find('atom:title', ns).text.strip()
        authors = [author.find('atom:name', ns).text for author in first_entry.findall('atom:author', ns)]
        published = first_entry.find('atom:published', ns).text
        
        print(f"\n=== 첫 번째 논문 정보 ===")
        print(f"제목: {title}")
        print(f"저자: {', '.join(authors[:3])}...")  # 처음 3명만 표시
        print(f"게시일: {published}")
        
except Exception as e:
    print(f"XML 파싱 오류: {e}")


=== XML 구조 분석 ===
루트 태그: {http://www.w3.org/2005/Atom}feed
총 entry 개수: 1

=== 첫 번째 논문 정보 ===
제목: Dynamic Backtracking
저자: M. L. Ginsberg...
게시일: 1993-08-01T00:00:00Z


In [11]:
# arXiv API 기본 사용법 3: 여러 카테고리에서 최신 논문 검색
from datetime import datetime, timedelta

# 여러 AI 카테고리 정의
ai_categories = ['cs.AI', 'cs.LG', 'cs.CL']
yesterday = datetime.now() - timedelta(days=1)

print(f"검색 대상 날짜: {yesterday.date()}")
print("=== 카테고리별 최신 논문 검색 ===")

for category in ai_categories:
    print(f"\n📚 {category} 카테고리:")
    
    # 각 카테고리별 최신 논문 5개 요청
    query_url = f'http://export.arxiv.org/api/query?search_query=cat:{category}&start=0&max_results=5&sortBy=submittedDate&sortOrder=descending'
    
    try:
        response = requests.get(query_url)
        root = ET.fromstring(response.content)
        
        ns = {
            'atom': 'http://www.w3.org/2005/Atom',
            'arxiv': 'http://arxiv.org/schemas/atom'
        }
        
        entries = root.findall('atom:entry', ns)
        recent_count = 0
        
        for entry in entries:
            published = entry.find('atom:published', ns).text
            pub_date = datetime.strptime(published[:10], '%Y-%m-%d')
            
            # 최근 2일 이내 논문만 카운트
            if (datetime.now() - pub_date).days <= 2:
                recent_count += 1
                if recent_count <= 2:  # 처음 2개만 제목 표시
                    title = entry.find('atom:title', ns).text.strip().replace('\n', ' ')
                    print(f"  - {title[:80]}...")
        
        print(f"  📊 최근 2일 내 논문 수: {recent_count}개")
        
        time.sleep(0.5)  # API 호출 간격 조절
        
    except Exception as e:
        print(f"  ❌ {category} 검색 오류: {e}")

print(f"\n✅ 기본 arXiv API 사용법 완료!")


검색 대상 날짜: 2025-06-09
=== 카테고리별 최신 논문 검색 ===

📚 cs.AI 카테고리:
  - StableMTL: Repurposing Latent Diffusion Models for Multi-Task Learning   from Pa...
  - Vision Transformers Don't Need Trained Registers...
  📊 최근 2일 내 논문 수: 5개

📚 cs.LG 카테고리:
  - StableMTL: Repurposing Latent Diffusion Models for Multi-Task Learning   from Pa...
  - Self Forcing: Bridging the Train-Test Gap in Autoregressive Video   Diffusion...
  📊 최근 2일 내 논문 수: 5개

📚 cs.CL 카테고리:
  - Play to Generalize: Learning to Reason Through Game Play...
  - Reinforcement Pre-Training...
  📊 최근 2일 내 논문 수: 5개

✅ 기본 arXiv API 사용법 완료!


### 1.2 arXiv 논문 수집 함수 정의

arXiv API를 사용하여 AI 관련 논문들을 수집하는 함수를 정의합니다.


In [12]:
def get_yesterday_arxiv_ai_papers():
    """어제 올라온 arXiv AI 관련 논문들을 가져오는 함수"""
    yesterday = datetime.now() - timedelta(days=1)
    
    # AI 관련 카테고리들
    ai_categories = [
        'cs.AI',  # Artificial Intelligence
        'cs.LG',  # Machine Learning
        'cs.CL',  # Computation and Language (NLP)
        'cs.CV',  # Computer Vision
        'cs.NE',  # Neural and Evolutionary Computing
        'stat.ML'  # Machine Learning (Statistics)
    ]
    
    all_papers = []
    
    for category in ai_categories:
        print(f"Fetching papers from {category}...")
        
        # arXiv API 쿼리 구성
        base_url = 'http://export.arxiv.org/api/query?'
        query = f'search_query=cat:{category}&start=0&max_results=100&sortBy=submittedDate&sortOrder=descending'
        url = base_url + query
        
        try:
            response = requests.get(url)
            response.raise_for_status()
            
            # XML 파싱
            root = ET.fromstring(response.content)
            
            # 네임스페이스 정의
            ns = {
                'atom': 'http://www.w3.org/2005/Atom',
                'arxiv': 'http://arxiv.org/schemas/atom'
            }
            
            # 각 논문 정보 추출
            for entry in root.findall('atom:entry', ns):
                published = entry.find('atom:published', ns).text
                pub_date = datetime.strptime(published[:10], '%Y-%m-%d')
                
                # 어제 올라온 논문만 필터링
                if pub_date.date() == yesterday.date():
                    paper_info = {
                        'title': entry.find('atom:title', ns).text.strip().replace('\\n', ' '),
                        'authors': [author.find('atom:name', ns).text for author in entry.findall('atom:author', ns)],
                        'summary': entry.find('atom:summary', ns).text.strip().replace('\\n', ' '),
                        'published': published,
                        'id': entry.find('atom:id', ns).text,
                        'category': category,
                        'link': entry.find('atom:id', ns).text.replace('http://arxiv.org/abs/', 'https://arxiv.org/abs/')
                    }
                    
                    primary_category = entry.find('arxiv:primary_category', ns)
                    if primary_category is not None:
                        paper_info['primary_category'] = primary_category.get('term')
                    
                    all_papers.append(paper_info)
            
            time.sleep(1)  # API 호출 간격 조절
            
        except Exception as e:
            print(f"Error fetching {category}: {e}")
            continue
    
    return all_papers


### 1.3 Document 변환 함수 정의

수집한 논문 데이터를 LangChain Document 객체로 변환하는 함수를 정의합니다.


In [13]:
def create_papers_documents(papers):
    """논문 리스트를 langchain Document 객체 리스트로 변환"""
    if not papers:
        print("No papers found for yesterday.")
        return []
    
    documents = []
    
    for paper in papers:
        # 저자 리스트를 문자열로 변환
        authors_str = ', '.join(paper['authors'])
        
        # 메타데이터 설정 (초록을 제외한 모든 정보)
        metadata = {
            'title': paper['title'],
            'authors': authors_str,
            'published': paper['published'],
            'id': paper['id'],
            'category': paper['category'],
            'primary_category': paper.get('primary_category', 'N/A'),
            'link': paper['link']
        }
        
        # Document 객체 생성 (page_content는 초록만)
        doc = Document(
            page_content=paper['summary'],
            metadata=metadata
        )
        
        documents.append(doc)
    
    # 중복 제거 (같은 논문이 여러 카테고리에 속할 수 있음)
    unique_documents = []
    seen_ids = set()
    
    for doc in documents:
        if doc.metadata['id'] not in seen_ids:
            unique_documents.append(doc)
            seen_ids.add(doc.metadata['id'])
    
    return unique_documents


### 1.4 논문 데이터 수집 실행

위에서 정의한 함수들을 사용하여 실제로 논문 데이터를 수집합니다.


In [14]:
# 논문 데이터 수집 및 Document 변환
papers = get_yesterday_arxiv_ai_papers()
documents = create_papers_documents(papers)
print(f"✅ {len(documents)}개의 논문이 Document 형태로 변환되었습니다.")

Fetching papers from cs.AI...
Fetching papers from cs.LG...
Fetching papers from cs.CL...
Fetching papers from cs.CV...
Fetching papers from cs.NE...
Fetching papers from stat.ML...
✅ 258개의 논문이 Document 형태로 변환되었습니다.


### 1.5 첫 번째 논문 확인

수집된 논문 중 첫 번째 논문의 내용을 확인해 보겠습니다.


In [15]:
# 첫 번째 논문 확인
if papers:
    print(f"첫 번째 논문: {papers[0]['title'][:100]}...")
    print(f"저자: {papers[0]['authors'][:3]}")  # 처음 3명의 저자만 표시
    print(f"카테고리: {papers[0]['category']}")
    print(f"초록 일부: {papers[0]['summary'][:200]}...")
else:
    print("수집된 논문이 없습니다.")

첫 번째 논문: StableMTL: Repurposing Latent Diffusion Models for Multi-Task Learning
  from Partially Annotated Sy...
저자: ['Anh-Quan Cao', 'Ivan Lopes', 'Raoul de Charette']
카테고리: cs.AI
초록 일부: Multi-task learning for dense prediction is limited by the need for extensive
annotation for every task, though recent works have explored training with
partial task labels. Leveraging the generalizat...


## 2. Qdrant 벡터 스토어 설정 및 문서 저장

수집한 arXiv 논문들을 Qdrant 벡터 스토어에 저장하여 Self-Query Retriever에서 사용할 수 있도록 준비합니다.

### 2.1 필요한 라이브러리 import


In [16]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

### 2.2 Qdrant 클라이언트 설정 및 컬렉션 생성

Qdrant 서버에 연결하고 논문 데이터를 저장할 컬렉션을 생성합니다.


In [ ]:
# Qdrant 클라이언트 설정
client = QdrantClient(host="localhost", port=6333)

# 컬렉션 생성 (이미 존재하는 경우 에러 무시)
try:
    client.create_collection(
        collection_name="arxiv_papers_0610",
        vectors_config={"dense": VectorParams(size=1024, distance=Distance.COSINE)}
    )
    print("✅ 'arxiv_papers' 컬렉션이 생성되었습니다.")
except Exception as e:
    print(f"컬렉션이 이미 존재하거나 생성 중 오류가 발생했습니다: {e}")

✅ 'arxiv_papers' 컬렉션이 생성되었습니다.


### 2.3 벡터 스토어 초기화 및 문서 저장

LangChain의 QdrantVectorStore를 초기화하고 논문 문서들을 임베딩하여 저장합니다.


In [19]:
# LangChain QdrantVectorStore 초기화
vectorstore = QdrantVectorStore(
    client=client,
    collection_name="arxiv_papers_0610",
    embedding=OllamaEmbeddings(model="bge-m3"),
    vector_name="dense"
)

print("🔄 논문 문서 임베딩 및 저장 중...")
vectorstore.add_documents(documents)
print(f"✅ {len(documents)}개 논문 문서가 벡터 저장소에 추가되었습니다.")


🔄 논문 문서 임베딩 및 저장 중...
✅ 258개 논문 문서가 벡터 저장소에 추가되었습니다.


### 2.4 저장된 데이터 확인

벡터 스토어에 저장된 논문 데이터를 간단한 검색으로 확인해봅니다.


In [20]:
# 간단한 유사도 검색 테스트
test_results = vectorstore.similarity_search("AI 에이전트", k=3)

print("검색 테스트 결과:")
for i, doc in enumerate(test_results, 1):
    print(f"\n{i}. 제목: {doc.metadata['title']}")
    print(f"   카테고리: {doc.metadata['category']}")
    print(f"   초록 일부: {doc.page_content[:150]}...")


검색 테스트 결과:

1. 제목: $τ^2$-Bench: Evaluating Conversational Agents in a Dual-Control
  Environment
   카테고리: cs.AI
   초록 일부: Existing benchmarks for conversational AI agents simulate single-control
environments, where only the AI agent can use tools to interact with the worl...

2. 제목: IntenTest: Stress Testing for Intent Integrity in API-Calling LLM Agents
   카테고리: cs.AI
   초록 일부: LLM agents are increasingly deployed to automate real-world tasks by invoking
APIs through natural language instructions. While powerful, they often s...

3. 제목: MCPWorld: A Unified Benchmarking Testbed for API, GUI, and Hybrid
  Computer Use Agents
   카테고리: cs.AI
   초록 일부: (M)LLM-powered computer use agents (CUA) are emerging as a transformative
technique to automate human-computer interaction. However, existing CUA
benc...


## 3. Self-Query Retriever 설정

이제 Self-Query Retriever를 설정하여 자연어 질의를 통해 메타데이터 기반 검색을 수행할 수 있도록 합니다.

### 3.1 메타데이터 필드 정보 정의

Self-Query Retriever가 이해할 수 있도록 논문 메타데이터의 각 필드에 대한 정보를 정의합니다.


In [21]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
# 메타데이터 필드 정보 정의
metadata_field_info = [
    AttributeInfo(
        name="title",
        description="논문의 제목",
        type="string",
    ),
    AttributeInfo(
        name="authors",
        description="논문의 저자들 (쉼표로 구분된 문자열)",
        type="string",
    ),
    AttributeInfo(
        name="category",
        description="논문의 arXiv 카테고리 (예: cs.AI, cs.LG, cs.CL, cs.CV, cs.NE, stat.ML)",
        type="string",
    ),
    AttributeInfo(
        name="primary_category",
        description="논문의 주요 카테고리",
        type="string",
    ),
    AttributeInfo(
        name="published",
        description="논문의 게시 날짜 (ISO 형식)",
        type="string",
    ),
    AttributeInfo(
        name="id",
        description="논문의 arXiv ID",
        type="string",
    ),
]

# 문서 내용에 대한 설명
document_content_description = "arXiv AI 관련 논문의 초록 (abstract)"

print("✅ 메타데이터 필드 정보가 정의되었습니다.")

✅ 메타데이터 필드 정보가 정의되었습니다.


### 3.2 LLM 모델 설정 및 Self-Query Retriever 생성

LLM을 설정하고 Self-Query Retriever를 생성합니다.


In [22]:
from langchain.chat_models import init_chat_model
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
load_dotenv()
# LLM 모델 설정 (Ollama 사용)
llm = init_chat_model("openai:gpt-4.1")
qwen= ChatOllama(
    model="qwen3",
    temperature=0
)
# Self-Query Retriever 생성
retriever = SelfQueryRetriever.from_llm(
    llm=qwen,
    vectorstore=vectorstore,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info,
    enable_limit=True,  # 결과 수 제한 기능 활성화
    verbose=True  # 디버깅을 위한 상세 출력
)

print("✅ Self-Query Retriever가 생성되었습니다.")


✅ Self-Query Retriever가 생성되었습니다.


## 4. Self-Query Retriever 실습

이제 Self-Query Retriever를 사용하여 다양한 자연어 질의를 수행해보겠습니다.

### 4.1 기본 검색 - 카테고리 기반 필터링


In [23]:
# 컴퓨터 비전 카테고리의 논문 검색
qwen_think="/no_think"
original_query = "컴퓨터 비전 카테고리에서 이미지 분할에 관한 논문을 찾아주세요"
query=f"{qwen_think} \n original_query:{original_query}"
print(f"질의: {original_query}")
print("=" * 50)

results = retriever.invoke(query)

print(f"검색 결과: {len(results)}개 논문")
for i, doc in enumerate(results, 1):
    print(f"\n{i}. 제목: {doc.metadata['title']}")
    print(f"   카테고리: {doc.metadata['category']} (주요: {doc.metadata['primary_category']})")
    print(f"   저자: {doc.metadata['authors'][:100]}...")
    print(f"   초록: {doc.page_content[:200]}...")

질의: 컴퓨터 비전 카테고리에서 이미지 분할에 관한 논문을 찾아주세요
검색 결과: 5개 논문

1. 제목: Text-guided multi-stage cross-perception network for medical image
  segmentation
   카테고리: cs.CV (주요: eess.IV)
   저자: Gaoyu Chen...
   초록: Medical image segmentation plays a crucial role in clinical medicine, serving
as a tool for auxiliary diagnosis, treatment planning, and disease monitoring,
thus facilitating physicians in the study a...

2. 제목: Cross-channel Perception Learning for H&E-to-IHC Virtual Staining
   카테고리: cs.CV (주요: cs.CV)
   저자: Hao Yang, JianYu Wu, Run Fang, Xuelian Zhao, Yuan Ji, Zhiyu Chen, Guibin He, Junceng Guo, Yang Liu, ...
   초록: With the rapid development of digital pathology, virtual staining has become
a key technology in multimedia medical information systems, offering new
possibilities for the analysis and diagnosis of pa...

3. 제목: Adaptive Blind Super-Resolution Network for Spatial-Specific and
  Spatial-Agnostic Degradations
   카테고리: cs.CV (주요: cs.CV)
   저자: Weilei Wen, Chunle Guo, Wenqi Ren, 

### 4.2 복합 조건 검색 - 저자와 주제 조합


In [24]:
# 특정 저자의 논문 중 딥러닝 관련 연구 검색
qwen_think="no_think"
original_query = "자연어 처리 카테고리에서 transformer나 attention에 관한 논문을 찾아주세요"
query=f"{qwen_think} \n original_query:{original_query}"

print(f"질의: {original_query}")
print("=" * 50)

results = retriever.invoke(query)

print(f"검색 결과: {len(results)}개 논문")
for i, doc in enumerate(results, 1):
    print(f"\n{i}. 제목: {doc.metadata['title']}")
    print(f"   카테고리: {doc.metadata['category']} (주요: {doc.metadata['primary_category']})")
    print(f"   저자: {doc.metadata['authors'][:100]}...")
    print(f"   초록: {doc.page_content[:200]}...")


질의: 자연어 처리 카테고리에서 transformer나 attention에 관한 논문을 찾아주세요
검색 결과: 4개 논문

1. 제목: Quantum Graph Transformer for NLP Sentiment Classification
   카테고리: cs.CL (주요: cs.CL)
   저자: Shamminuj Aktar, Andreas Bärtschi, Abdel-Hameed A. Badawy, Stephan Eidenbenz...
   초록: Quantum machine learning is a promising direction for building more efficient
and expressive models, particularly in domains where understanding complex,
structured data is critical. We present the Qu...

2. 제목: Learning to Focus: Causal Attention Distillation via Gradient-Guided
  Token Pruning
   카테고리: cs.CL (주요: cs.CL)
   저자: Yiju Guo, Wenkai Yang, Zexu Sun, Ning Ding, Zhiyuan Liu, Yankai Lin...
   초록: Large language models (LLMs) have demonstrated significant improvements in
contextual understanding. However, their ability to attend to truly critical
information during long-context reasoning and ge...

3. 제목: Intent Matters: Enhancing AI Tutoring with Fine-Grained Pedagogical
  Intent Annotation
   카테고리: cs.CL (주요: cs.CL)
   저자: K

### 4.3 결과 수 제한 검색


In [25]:
# 결과 수 제한과 함께 검색
qwen_think="/no_think"
original_query = "AI 카테고리에서 강화학습에 관한 논문 3개만 찾아주세요"
query=f"{qwen_think} \n original_query:{original_query}"

print(f"질의: {original_query}")
print("=" * 50)

results = retriever.invoke(query)

print(f"검색 결과: {len(results)}개 논문")
for i, doc in enumerate(results, 1):
    print(f"\n{i}. 제목: {doc.metadata['title']}")
    print(f"   카테고리: {doc.metadata['category']} (주요: {doc.metadata['primary_category']})")
    print(f"   저자: {doc.metadata['authors'][:100]}...")
    print(f"   초록: {doc.page_content[:200]}...")


질의: AI 카테고리에서 강화학습에 관한 논문 3개만 찾아주세요
검색 결과: 3개 논문

1. 제목: Curriculum Learning With Counterfactual Group Relative Policy Advantage
  For Multi-Agent Reinforcement Learning
   카테고리: cs.AI (주요: cs.AI)
   저자: Weiqiang Jin, Hongyang Du, Guizhong Liu, Dong In Kim...
   초록: Multi-agent reinforcement learning (MARL) has achieved strong performance in
cooperative adversarial tasks. However, most existing methods typically train
agents against fixed opponent strategies and ...

2. 제목: Learning What Reinforcement Learning Can't: Interleaved Online
  Fine-Tuning for Hardest Questions
   카테고리: cs.AI (주요: cs.AI)
   저자: Lu Ma, Hao Liang, Meiyi Qiang, Lexiang Tang, Xiaochen Ma, Zhen Hao Wong, Junbo Niu, Chengyu Shen, Ru...
   초록: Recent advances in large language model (LLM) reasoning have shown that
sophisticated behaviors such as planning and self-reflection can emerge through
reinforcement learning (RL). However, despite th...

3. 제목: REMoH: A Reflective Evolution of Multi-objective Heuristics appro

### 4.4 다양한 질의 유형 실험

다른 형태의 자연어 질의들을 시도해보겠습니다.


In [26]:
# 여러 가지 질의 유형 테스트
test_queries = [
    "인공지능 카테고리의 논문들 중에서 robot이나 robotics가 포함된 것들을 찾아주세요",
    "generative model이나 diffusion에 관한 연구를 찾아주세요",
    "cs.CV 카테고리에서 object detection 관련 논문들을 보여주세요"
]

for original_query in test_queries:
    qwen_think="/no_think"
    query=f"{qwen_think} \n original_query:{original_query}"
    
    print(f"\n🔍 질의: {original_query}")
    print("-" * 80)
    try:
        results = retriever.invoke(query)
        print(f"검색 결과: {len(results)}개 논문")
        
        for i, doc in enumerate(results[:2], 1):  # 각 질의당 최대 2개 결과만 표시
            print(f"\n{i}. 제목: {doc.metadata['title']}")
            print(f"   카테고리: {doc.metadata['category']}")
            print(f"   초록: {doc.page_content[:150]}...")
            
    except Exception as e:
        print(f"검색 중 오류 발생: {e}")
    
    print("\n" + "="*100)



🔍 질의: 인공지능 카테고리의 논문들 중에서 robot이나 robotics가 포함된 것들을 찾아주세요
--------------------------------------------------------------------------------
검색 중 오류 발생: Parsing text
<think>

</think>

```json
{
    "query": "robot",
    "filter": "and(like(\"primary_category\", \"cs.AI\"), or(like(\"title\", \"robot\"), like(\"title\", \"robotics\")), like(\"authors\", \"robot\"), like(\"authors\", \"robotics\")))",
    "limit": 0
}
```
 raised following error:
Unexpected token Token('RPAR', ')') at line 1, column 149.
Expected one of: 
	* $END

For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


🔍 질의: generative model이나 diffusion에 관한 연구를 찾아주세요
--------------------------------------------------------------------------------
검색 결과: 4개 논문

1. 제목: FunDiff: Diffusion Models over Function Spaces for Physics-Informed
  Generative Modeling
   카테고리: cs.LG
   초록: Recent advances in generative modeling -- particularly diffusion models and
flow matching --